[![ko-fi](https://ko-fi.com/img/githubbutton_sm.svg)](https://ko-fi.com/Q5Q811R5YI)  
# Apollo-Colab-Inference [![Open In Github](https://img.shields.io/badge/github-code-green)](https://github.com/jarredou/Apollo-Colab-Inference/)  


*Original work [Apollo: Band-sequence Modeling for High-Quality Music Restoration in Compressed Audio](https://github.com/JusperLee/Apollo)*  

The model was trained to restore/enhance lossy mp3 audio with bitrate <= 128 kbps.  
<br>
___  
*changelog:*
   
<font size=2>**v0.2**  
<font size=2>- added overlap feature  
<font size=2>- new inference.py created for easier local CLI use  

<font size=2>**v0.1**  
<font size=2>- added chunking for long audio inputs  
<font size=2>- ~~added "dual mono" processing for stereo audio input (processing each channel independently)~~

In [ ]:
%%capture --no-stderr
#@markdown #Install
%cd /content/
!git clone https://github.com/JusperLee/Apollo.git && cd Apollo

!mkdir /content/Apollo/model
%cd /content/Apollo/model
!wget https://huggingface.co/JusperLee/Apollo/resolve/main/pytorch_model.bin

!rm -rf '/content/Apollo/inference.py'
%cd /content/Apollo
!wget 'https://raw.githubusercontent.com/jarredou/Apollo-Colab-Inference/main/inference.py'

!pip install omegaconf

In [ ]:
%cd /content/Apollo
#@markdown #Inference
input_file_path = '/content/input.wav' #@param {type:"string"}
output_file_path = '/content/output.wav' #@param {type:"string"}
chunk_size = 3 #@param {type:"slider", min:3, max:25, step:1}
overlap = 2 #@param {type:"slider", min:2, max:10, step:1}

!python inference.py \
    --in_wav '{input_file_path}' \
    --out_wav '{output_file_path}' \
    --chunk_size {chunk_size} \
    --overlap {overlap}